In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Mitochondrial_Disorders"
cohort = "GSE42986"

# Input paths
in_trait_dir = "../../input/GEO/Mitochondrial_Disorders"
in_cohort_dir = "../../input/GEO/Mitochondrial_Disorders/GSE42986"

# Output paths
out_data_file = "../../output/preprocess/Mitochondrial_Disorders/GSE42986.csv"
out_gene_data_file = "../../output/preprocess/Mitochondrial_Disorders/gene_data/GSE42986.csv"
out_clinical_data_file = "../../output/preprocess/Mitochondrial_Disorders/clinical_data/GSE42986.csv"
json_path = "../../output/preprocess/Mitochondrial_Disorders/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE42986_family.soft.gz', 'GSE42986_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Mitochondrial_Disorders/GSE42986/GSE42986_family.soft.gz
Matrix file: ../../input/GEO/Mitochondrial_Disorders/GSE42986/GSE42986_series_matrix.txt.gz
Background Information:
!Series_title	"Transcriptome profiling in human primary mitochondrial respiratory chain disease"
!Series_summary	"Primary mitochondrial respiratory chain (RC) diseases are heterogeneous in etiology and manifestations but collectively impair cellular energy metabolism.  To identify a common cellular response to RC disease, systems biology level transcriptome investigations were performed in human RC disease skeletal muscle and fibroblasts.  Global transcriptional and post-transcriptional dysregulation in a tissue-specific fashion was identified across diverse RC complex and genetic etiologies.  RC disease muscle was characterized by decreased transcription of cytosolic ribosomal proteins to reduce energy-in

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background info, this dataset contains transcriptome profiling data 
# from Affymetrix Human Exon 1.0ST microarray, so it's suitable for our analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait: Mitochondrial Disorders are reflected in respiratory chain deficiency
trait_row = 1  # Key for respiratory chain complex deficiency

# For age: available in key 3
age_row = 3

# For gender: available in key 2
gender_row = 2

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """
    Convert respiratory chain complex deficiency into binary (0 = No deficiency, 1 = Has deficiency)
    """
    if not value or ":" not in value:
        return None
    
    value = value.split(":", 1)[1].strip()
    
    # No deficiency cases
    if "No Respiratory Chain Complex Deficiency" in value:
        return 0
    # All other cases with respiratory chain complex deficiency
    elif "Complex" in value or "Complexes" in value or "mtDNA depletion" in value:
        return 1
    # If we can't determine
    else:
        return None

def convert_age(value):
    """
    Convert age to continuous value (float)
    """
    if not value or ":" not in value:
        return None
    
    value = value.split(":", 1)[1].strip()
    
    if value.lower() == "not obtained":
        return None
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """
    Convert gender to binary (0 = Female, 1 = Male)
    """
    if not value or ":" not in value:
        return None
    
    value = value.split(":", 1)[1].strip()
    
    if value.upper() == "F":
        return 0
    elif value.upper() == "M":
        return 1
    else:
        return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # The sample characteristics dictionary from the output shows unique values
    # We need to read the actual data from the series matrix file
    
    # First, let's load the matrix file to extract the clinical data
    matrix_file = f"{in_cohort_dir}/GSE42986_series_matrix.txt.gz"
    
    # Read the matrix file to get clinical data
    with gzip.open(matrix_file, 'rt') as file:
        lines = []
        # Read the file until we reach the sample data
        for line in file:
            if line.startswith('!Sample_'):
                lines.append(line.strip())
            # Stop when we reach the table data
            if line.startswith('!series_matrix_table_begin'):
                break
    
    # Parse the sample information into a proper DataFrame
    sample_ids = []
    sample_data = {}
    
    for line in lines:
        if line.startswith('!Sample_geo_accession'):
            sample_ids = line.split('\t')[1:]
            for i in range(len(sample_ids)):
                sample_data[i] = {}
                
        elif line.startswith('!Sample_characteristics_ch1'):
            values = line.split('\t')[1:]
            for i, value in enumerate(values):
                if i < len(sample_ids):
                    # Extract the key (before the :)
                    if ':' in value:
                        key_part = value.split(':', 1)[0].strip()
                        # Store the characteristic under the appropriate row
                        if key_part == 'respiratory chain complex deficiency' or key_part == 'respiratory chain complex deficiency in muscle':
                            if 1 not in sample_data[i]:
                                sample_data[i][1] = []
                            sample_data[i][1].append(value)
                        elif key_part == 'gender':
                            if 2 not in sample_data[i]:
                                sample_data[i][2] = []
                            sample_data[i][2].append(value)
                        elif key_part == 'age (years)':
                            if 3 not in sample_data[i]:
                                sample_data[i][3] = []
                            sample_data[i][3].append(value)
    
    # Create a DataFrame from the parsed data
    clinical_data_rows = []
    for i in range(len(sample_ids)):
        row_data = {}
        for row_key in [1, 2, 3]:  # trait_row, gender_row, age_row
            if row_key in sample_data[i] and sample_data[i][row_key]:
                row_data[row_key] = sample_data[i][row_key][0]  # Take the first value
            else:
                row_data[row_key] = None
        clinical_data_rows.append(row_data)
    
    clinical_data = pd.DataFrame(clinical_data_rows)
    
    # Extract clinical features using the library function
    selected_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(selected_features)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{1: [nan, nan, nan], 2: [nan, nan, nan], 3: [nan, nan, nan]}
Clinical data saved to ../../output/preprocess/Mitochondrial_Disorders/clinical_data/GSE42986.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.

Gene data extraction result:
Number of rows: 20788
First 20 gene/probe identifiers:
Index(['100009676_at', '10000_at', '10001_at', '10002_at', '100033416_at',
       '100033422_at', '100033423_at', '100033424_at', '100033425_at',
       '100033426_at', '100033428_at', '100033431_at', '100033434_at',
       '100033436_at', '100033438_at', '100033439_at', '100033444_at',
       '100033800_at', '100033806_at', '100033819_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers format: '100009676_at', '10000_at', etc.
# These appear to be Affymetrix probe IDs (with the "_at" suffix), 
# not standard human gene symbols.
# Affymetrix probe IDs need to be mapped to standard gene symbols for analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1_at', '2_at', '9_at', '10_at', '12_at'], 'Gene_ID': ['1', '2', '9', '10', '12'], 'ORF': ['A1BG', 'A2M', 'NAT1', 'NAT2', 'SERPINA3'], 'Symbol': ['A1BG', 'A2M', 'NAT1', 'NAT2', 'SERPINA3'], 'Chromosome': ['19', '12', '8', '8', '14'], 'RefSeq_ID': ['NM_130786;NP_570602', 'NM_000014;NP_000005', 'NM_000662;NM_001160170;NM_001160171;NM_001160172;NM_001160173;NM_001160174;NM_001160175;NM_001160176;NM_001160179;NP_000653;NP_001153642;NP_001153643;NP_001153644;NP_001153645;NP_001153646;NP_001153647;NP_001153648;NP_001153651', 'NM_000015;NP_000006', 'NM_001085;NP_001076'], 'Num_Probes': [47.0, 167.0, 74.0, 20.0, 56.0], 'Full_Name': ['alpha-1-B glycoprotein', 'alpha-2-macroglobulin', 'N-acetyltransferase 1 (arylamine N-acetyltransferase)', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'serpin peptidase inhibitor, clade A (alpha-1 antiproteinase, antitrypsin), member 3']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene annotation correspond to probe IDs and gene symbols
# From the preview, we can see:
# - 'ID' contains values like '1_at', '2_at' which match the format of our gene expression data
# - 'Symbol' contains standard gene symbols like 'A1BG', 'A2M'

# 2. Create a mapping dataframe with the ID (probe) and Symbol (gene) columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

print("Gene mapping preview (first 5 rows):")
print(gene_mapping.head())
print(f"Total number of mappings: {len(gene_mapping)}")

# 3. Apply the gene mapping to convert probe-level data to gene-level expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

print("\nGene expression data after mapping:")
print(f"Number of genes: {len(gene_data)}")
print("First few genes:")
print(gene_data.index[:10])

# Normalize gene symbols to handle gene synonym variations
gene_data = normalize_gene_symbols_in_index(gene_data)
print("\nGene expression data after normalization:")
print(f"Number of genes after normalization: {len(gene_data)}")
print("First few normalized genes:")
print(gene_data.index[:10])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Gene mapping preview (first 5 rows):
      ID      Gene
0   1_at      A1BG
1   2_at       A2M
2   9_at      NAT1
3  10_at      NAT2
4  12_at  SERPINA3
Total number of mappings: 20788

Gene expression data after mapping:
Number of genes: 19870
First few genes:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT',
       'AA06', 'AAA1'],
      dtype='object', name='Gene')



Gene expression data after normalization:
Number of genes after normalization: 19636
First few normalized genes:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AA06',
       'AAA1', 'AAAS'],
      dtype='object', name='Gene')



Gene expression data saved to ../../output/preprocess/Mitochondrial_Disorders/gene_data/GSE42986.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# (This was already done in step 6, but we'll keep this to ensure the correct workflow)
print(f"Shape of gene data after normalization: {gene_data.shape}")

# 2. Extract clinical data from the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print out the sample characteristics to verify available rows
characteristics_dict = get_unique_values_by_row(clinical_data)
print("Sample characteristics dictionary:")
print(characteristics_dict)

# Define conversion functions for the clinical features based on the actual data
# Reusing the correct conversion functions from step 2
def convert_trait(value):
    """
    Convert respiratory chain complex deficiency into binary (0 = No deficiency, 1 = Has deficiency)
    """
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # No deficiency cases
    if "No Respiratory Chain Complex Deficiency" in value:
        return 0
    # All other cases with respiratory chain complex deficiency
    elif "Complex" in value or "Complexes" in value or "mtDNA depletion" in value:
        return 1
    # If we can't determine
    else:
        return None

def convert_age(value):
    """
    Convert age to continuous value (float)
    """
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.lower() == "not obtained":
        return None
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """
    Convert gender to binary (0 = Female, 1 = Male)
    """
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.upper() == "F":
        return 0
    elif value.upper() == "M":
        return 1
    else:
        return None

# Create the clinical dataframe using the identified rows from step 2
try:
    clinical_df = geo_select_clinical_features(
        clinical_data,
        trait=trait,  # Using the predefined trait variable
        trait_row=1,  # Row for respiratory chain complex deficiency
        convert_trait=convert_trait,
        gender_row=2,  # Gender information
        convert_gender=convert_gender,
        age_row=3,    # Age information
        convert_age=convert_age
    )
    
    print("Clinical data preview:")
    print(preview_df(clinical_df.T))  # Transpose for better viewing
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Saved clinical data to {out_clinical_data_file}")
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
    print(f"Shape of linked data: {linked_data.shape}")
    
    # 4. Handle missing values in the linked data
    linked_data_cleaned = handle_missing_values(linked_data, trait)
    print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")
    
    # 5. Check if the trait and demographic features are biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)
    
    # 6. Validate the dataset and save cohort information
    note = "Dataset contains gene expression data from human samples with mitochondrial respiratory chain disease. The trait variable represents respiratory chain complex deficiency (0=No deficiency, 1=Has deficiency)."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_trait_biased,
        df=unbiased_linked_data,
        note=note
    )
    
    # 7. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Saved processed linked data to {out_data_file}")
    else:
        print("Dataset validation failed. Final linked data not saved.")
        
except Exception as e:
    print(f"Error in processing clinical data: {e}")
    # If we failed to extract clinical data, update the cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=None,
        df=pd.DataFrame(),
        note="Failed to extract clinical data. Gene expression data is available but clinical feature extraction failed."
    )
    print("Dataset validation failed due to missing or corrupted clinical data. Only gene data saved.")

Shape of gene data after normalization: (19636, 53)
Sample characteristics dictionary:
{0: ['tissue: Skeletal muscle', 'tissue: fibroblast cell line'], 1: ['respiratory chain complex deficiency: No Respiratory Chain Complex Deficiency', 'respiratory chain complex deficiency: Complexes I and III', 'respiratory chain complex deficiency: Complex IV', 'respiratory chain complex deficiency: Complexes II and III', 'respiratory chain complex deficiency: Not measured; 87% mtDNA depletion in muscle', 'respiratory chain complex deficiency: Complex IV; 70% mtDNA depletion in liver', 'respiratory chain complex deficiency: Complex IV; 93% mtDNA depletion in muscle', 'respiratory chain complex deficiency: Complexes I and IV', 'respiratory chain complex deficiency: Complex I', 'respiratory chain complex deficiency: Complex I and IV', 'respiratory chain complex deficiency in muscle: Not Determined', 'respiratory chain complex deficiency in muscle: Complex I+III Deficiency', 'respiratory chain complex 

Shape of linked data after handling missing values: (46, 19639)
For the feature 'Mitochondrial_Disorders', the least common label is '1.0' with 21 occurrences. This represents 45.65% of the dataset.
The distribution of the feature 'Mitochondrial_Disorders' in this dataset is fine.

Quartiles for 'Age':
  25%: 1.0
  50% (Median): 5.0
  75%: 19.0
Min: 0.2
Max: 72.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 16 occurrences. This represents 34.78% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Saved processed linked data to ../../output/preprocess/Mitochondrial_Disorders/GSE42986.csv
